In [14]:
INPUT_FILE_NAME = 'log_sample_with_answers_full.csv'
#'small_log.csv'
#'log_sample.csv'
LABEL_TIME_PART = 0.2

In [15]:
import csv
import numpy as np
from tqdm import tqdm
import operator
import json

In [16]:
class Event:
    def __init__(self, timestamp, group_id, device_id, event_id, event_count):
        self.timestamp = int(timestamp)
        self.group_id = group_id
        self.event_id = event_id
        self.device_id = device_id
        if event_count:
            self.count = int(event_count)
        else:
            self.count = 0

In [17]:
def check_group_event_id(group_id, event_id):
    return group_id == 'actions' and event_id == 'action.invoked'

In [22]:
def read_from_file_to_event_list():
    events = []
    isFirst = True
    with open(INPUT_FILE_NAME, 'r') as fin:
        for row in tqdm(csv.reader(fin, delimiter=',')):
            if isFirst:
                isFirst = False
                continue
            if (row[12].split('.')[0]) and check_group_event_id(row[5], row[10]):
                event_info = json.loads(row[11])
                group_id = 'actions.action.invoked'
                event_id = event_info["action_id"]
                events.append(Event(row[3], group_id, row[7], event_id, row[12].split('.')[0]))
    return events
            

In [23]:
events = read_from_file_to_event_list()

10857583it [02:20, 77017.81it/s]


In [24]:
def get_user_to_time_period(events):
    user_to_min_time = {}
    user_to_max_time = {}
    labels = {}

    for event in tqdm(events):
        if event.device_id not in user_to_min_time.keys():
            user_to_min_time[event.device_id] = event.timestamp
        else:
            user_to_min_time[event.device_id] = min(user_to_min_time[event.device_id], event.timestamp)
        if event.device_id not in user_to_max_time.keys():
            user_to_max_time[event.device_id] = event.timestamp
        else:
            user_to_max_time[event.device_id] = max(user_to_max_time[event.device_id], event.timestamp)

    user_to_time_period = {}
    
    for user in user_to_min_time.keys():
        min_time = user_to_min_time[user]
        max_time = user_to_max_time[user]
        user_to_time_period[user] = int((max_time - min_time) / 1000 /60/60/24)
    return user_to_time_period  
        
    

In [25]:
user_to_time_period = get_user_to_time_period(events).values()

100%|██████████| 2379192/2379192 [00:05<00:00, 439862.96it/s]


In [26]:
print(len(user_to_time_period))

1636


In [27]:
period_to_count = np.zeros(100)
for period in user_to_time_period:
    period_to_count[period] += 1


In [28]:
for i in range(len(period_to_count)):
    print(str(i) + " days: " + str(int(period_to_count[i])) + " users")

0 days: 665 users
1 days: 134 users
2 days: 91 users
3 days: 88 users
4 days: 80 users
5 days: 53 users
6 days: 55 users
7 days: 53 users
8 days: 47 users
9 days: 40 users
10 days: 25 users
11 days: 38 users
12 days: 21 users
13 days: 18 users
14 days: 25 users
15 days: 22 users
16 days: 16 users
17 days: 23 users
18 days: 23 users
19 days: 13 users
20 days: 20 users
21 days: 31 users
22 days: 18 users
23 days: 13 users
24 days: 19 users
25 days: 5 users
26 days: 0 users
27 days: 0 users
28 days: 0 users
29 days: 0 users
30 days: 0 users
31 days: 0 users
32 days: 0 users
33 days: 0 users
34 days: 0 users
35 days: 0 users
36 days: 0 users
37 days: 0 users
38 days: 0 users
39 days: 0 users
40 days: 0 users
41 days: 0 users
42 days: 0 users
43 days: 0 users
44 days: 0 users
45 days: 0 users
46 days: 0 users
47 days: 0 users
48 days: 0 users
49 days: 0 users
50 days: 0 users
51 days: 0 users
52 days: 0 users
53 days: 0 users
54 days: 0 users
55 days: 0 users
56 days: 0 users
57 days: 0 use

In [29]:
def get_user_to_time_periods(events):
    user_to_timestamps = {}

    for event in tqdm(events):
        if event.device_id in user_to_timestamps.keys():
            user_to_timestamps[event.device_id].append(event.timestamp)
        else:
            user_to_timestamps[event.device_id] = [event.timestamp]
      
    user_to_time_periods = {}
    for user in tqdm(user_to_timestamps.keys()):
        user_to_timestamps[user] = sorted(user_to_timestamps[user])
        user_to_time_periods[user] = []
        for i in range(len(user_to_timestamps[user]) - 1):
            diff = int((user_to_timestamps[user][i + 1] - user_to_timestamps[user][i]) / 1000 / 60)
            if diff > 0:
                user_to_time_periods[user].append(diff)
    return user_to_time_periods
    

In [30]:
user_to_time_periods = get_user_to_time_periods(events)

100%|██████████| 1636/1636 [00:02<00:00, 721.92it/s] 


In [31]:
activity = []
for value in user_to_time_periods.values():
    cnt = 0
    for elem in value:
        if elem <= 30:
            cnt += elem
        else:
            #print("active(" + str(cnt), end=') ')
            activity.append(cnt)
            cnt = 0
            #print("stopped(" + str(elem), end=') ')
            #print()
    #print("active(" + str(cnt), end=') ')
    activity.append(cnt)  
#print(activity)
x = np.percentile(activity, 95)
print(x)

x1 = np.mean(activity)
print(x1)

x2 = np.median(activity)
print(x2)
    #print("END")

100.44999999999891
26.965527760829776
13.0


In [32]:
import sys

In [33]:
def get_user_to_action_cnt(events):
    user_to_cnt = {}
    for event in tqdm(events):
        if event.device_id not in user_to_cnt.keys():
            user_to_cnt[event.device_id] = sys.getsizeof(event.timestamp) + sys.getsizeof(event.group_id) + sys.getsizeof(event.event_id)  
        else:
            user_to_cnt[event.device_id] += sys.getsizeof(event.timestamp) + sys.getsizeof(event.group_id) + sys.getsizeof(event.event_id)   
    return max(user_to_cnt.items(), key=operator.itemgetter(1))

In [34]:
get_user_to_action_cnt(events)

100%|██████████| 2379192/2379192 [00:04<00:00, 530081.53it/s]


('270319163e52f29-0137-470f-b0d0-a8651264c103', 9357234)

In [35]:
44679581 / 1024 / 1024

42.609768867492676